In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My Drive/Colab Notebooks/Classification_Bankruptcy/5조 

/content/gdrive/My Drive/Colab Notebooks/Classification_Bankruptcy/5조


In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. Deleting useless variables using correlation 

In [4]:
df = pd.read_csv("imputed_final.csv")
df.head()

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,Attr11,Attr12,Attr13,Attr14,Attr16,Attr18,Attr21,Attr22,Attr23,Attr24,Attr25,Attr26,Attr27,Attr29,Attr30,Attr32,Attr33,Attr35,Attr36,Attr38,Attr39,Attr40,Attr43,Attr45,Attr47,Attr48,Attr49,Attr50,Attr51,Attr52,Attr53,Attr54,Attr56,Attr58,Attr59,Attr60,Attr61,Attr63,Attr64,class
0,-0.024558,0.30051,0.10733,1.975914,34.382,-0.026711,-0.020067,2.25190,1.06670,0.67673,-0.020067,-0.156456,0.025634,-0.020067,0.057548,-0.020067,1.08530,0.00866,-0.023022,-0.019018,0.67673,0.042607,0.13691,4.5645,0.44185,74.015,4.9315,0.00866,0.69446,0.84898,0.008118,0.026591,125.6100,-0.49620,28.560,-0.028702,-0.026907,0.783960,0.12826,0.20278,0.88530,1.1106,0.062497,0.93750,0.288165,13.6320,3.6930,8.316700,0.88260,0.0
1,0.639180,0.00000,1.00000,5006.666667,29.678,-1.139300,0.760520,1.09245,1.11850,1.00000,0.760520,2328.019515,0.679940,0.760520,0.222805,0.760520,1.04790,0.76040,0.571462,0.600720,-0.50015,0.203820,1.01190,1.2218,0.00000,0.000,4.4510,0.76040,1.11850,1.00000,0.679839,0.177130,9.6341,0.23541,0.000,0.760400,0.679839,27695.282739,0.00000,0.00000,1.20865,1.3747,0.679830,0.32014,0.000000,9.5619,37.8860,3423.828207,4.01995,0.0
2,-0.440760,0.47723,0.19260,1.467776,34.555,0.000000,-0.440760,1.09540,1.51860,0.52277,-0.429190,-1.070611,-0.240740,-0.440760,-0.766050,-0.440760,0.50040,-0.42986,-0.290241,0.744700,0.46954,-0.766050,-37.14700,2.9728,0.31392,76.670,4.7606,-0.44130,1.51860,0.58832,-0.290597,0.029665,142.3100,0.23541,0.000,-0.505040,-0.332569,1.266200,0.41169,0.21006,1.32110,1.4867,-0.290590,1.28710,0.125371,9.5619,2.5649,3.688698,3.83760,0.0
3,0.119520,0.17679,0.63546,4.933282,117.650,0.251540,0.148750,4.58260,1.12630,0.81017,0.148750,0.841394,0.131500,0.148750,1.189400,0.148750,0.97491,0.14495,0.106117,0.315270,0.81017,1.024100,1.02090,5.4536,-0.18039,45.450,8.0308,0.14495,1.62200,0.81017,0.128696,2.702300,76.3540,0.66747,46.033,0.083425,0.074070,4.594400,0.17679,0.12452,4.31510,4.3151,0.112110,0.88789,0.016095,8.9302,10.2870,6.370835,8.51690,0.0
4,0.230400,0.27095,0.23762,2.574483,-26.928,0.617540,0.282690,2.69070,0.80411,0.72905,0.284330,1.873236,0.390160,0.282690,1.157900,0.282690,1.44160,0.28433,0.286528,0.616550,0.62819,0.964870,173.05000,7.2973,0.33696,126.760,3.4578,0.36955,0.80411,0.84909,0.459576,1.102400,100.8400,2.27140,70.950,0.253300,0.315007,1.433900,0.15091,0.28920,1.19230,1.3886,0.459570,0.62320,0.164653,7.9274,6.6605,5.328408,1.31500,0.0


In [5]:
df.shape

(6835, 50)

In [6]:
df.columns[24]

'Attr30'

In [7]:
cor = df.iloc[:, :-1].corr()
cor.shape

(49, 49)

변수끼리 상관계수가 0.9가 넘어가면 일단 삭제해보자

In [0]:

def cor_high(df):
    high_correlated = []
    for attr, rows in df.iterrows():
        high = [(attr, rows.index[idx], round(row, 3)) for idx, row in enumerate(rows) if attr != rows.index[idx] and row > 0.9 \
                and ((rows.index[idx], attr, round(row, 3)) not in high_correlated)]
        high_correlated.extend(high)
    return high_correlated

In [9]:
high_correlated_cols = cor_high(cor)
print(high_correlated_cols)

[('Attr1', 'Attr7', 0.991), ('Attr1', 'Attr11', 0.968), ('Attr1', 'Attr14', 0.991), ('Attr7', 'Attr11', 0.98), ('Attr7', 'Attr14', 1.0), ('Attr8', 'Attr16', 0.93), ('Attr8', 'Attr26', 0.926), ('Attr11', 'Attr14', 0.98), ('Attr16', 'Attr26', 0.999), ('Attr16', 'Attr33', 0.931), ('Attr26', 'Attr33', 0.93), ('Attr32', 'Attr52', 0.978)]


타겟변수와 상관계수가 더 작은애들을 삭제하자

In [10]:
drop_cols = set()

for col1, col2, _ in high_correlated_cols:
    cor1, cor2 = pd.concat([df[col1], df['class']], axis=1).corr(), pd.concat([df[col2], df['class']], axis=1).corr()
    cor1, cor2 = cor1.iloc[0, 1], cor.iloc[0, 1]
    drop_cols.add(col1 if  abs(cor1) <  abs(cor2) else col2)

print(drop_cols)

{'Attr1', 'Attr16', 'Attr7', 'Attr8', 'Attr32', 'Attr26', 'Attr11'}


In [11]:
df.drop(drop_cols, axis=1, inplace=True)
print(df.shape) # 6 columns are removed

(6835, 43)


# 2. Additional Dimensionality Reduction
[link](https://scikit-learn.org/stable/modules/feature_selection.html)

In [0]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [13]:
X[X > 0].dropna(how='any', axis='columns').shape

(6835, 2)

다 양수인 컬럼이 2개밖에 없어서 chi2 검정은 못할 것 같습니다.. <br>
- F-검정을 활용한 feature selection -> k개를 조정해가면서 몇개 정도로 하는게 맞는지?
- L1-based feature selection
- Tree-based feature selection
- PCA, FA를 활용한 차원축소 <br>
<b> 정리하자면 우리 데이터에 맞는 모델링 기법을 하나 정한 후 위의 4가지 방법을 다 실행해보고 차원축소를 진행한 후, <br>
앙상블 기법으로(하이퍼파라미터 튜닝 물론 선행) 모델을 만들어보자! </b> <br>
우리가 배운 classification 기법: Logistic Regression, LDA, Support Vector, Trees